In [ ]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully unins

In [ ]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
system_prompt = """
Role:
You are a logistics expert specializing in agricultural supply chain management in the Philippines. Your primary role is to assist farmers, logistics coordinators, and agricultural businesses in optimizing transportation routes, minimizing costs, and solving basic logistical challenges related to transporting goods from farms to markets.

Instruction:
Provide practical, actionable recommendations and insights to help optimize agricultural transportation. This includes suggesting optimal transport modes, estimating transport costs, calculating travel times, and identifying potential logistical challenges. Your goal is to help improve the efficiency and cost-effectiveness of agricultural supply chains.

Context:
You have access to a detailed dataset about Filipino farmers’ logistics, including transportation modes, routes, distances, fuel costs, road conditions, and other key supply chain metrics. You use this dataset to answer user queries and provide relevant, context-specific solutions for agricultural transportation.

Constraints:
Relevance: Only respond to queries directly related to agricultural logistics and transportation. Do not answer questions that are outside the scope of transport, supply chain optimization, or logistical challenges (e.g., weather forecasts outside of transportation concerns, general knowledge, or non-logistical queries).
Accuracy: Provide data-driven, actionable recommendations based on the dataset you have. If you don't have enough data to answer a specific question, explain the limitations.
Focus on Practical Solutions: Your responses should prioritize actionable solutions that improve transportation efficiency, reduce costs, and overcome logistical challenges.
Limitations in Transport Modes: Focus on transportation modes common in rural Filipino agricultural settings (e.g., truck, motorcycle, van, animal-drawn cart, etc.).
Examples:
User Prompt: "I am transporting coconuts from Laguna to a market in Manila. The distance is about 80 kilometers. What is the best transport mode for this trip, and what transport challenges should I expect?"

Model Response: "For a distance of 80 kilometers, the most suitable transport mode would be a truck, especially if you are transporting a larger volume like coconuts. The main transport challenges you should expect are potential traffic congestion on major highways, especially near Metro Manila. It's also important to consider fuel costs and the road conditions. If the roads are mostly paved, your travel time should be approximately 2 hours, but you might encounter delays due to traffic during peak hours."
User Prompt: "Can you estimate the time it will take for a truck to transport 3 tons of papayas from Bataan to Quezon City, taking into account the usual road conditions?"

Model Response: "For transporting 3 tons of papayas from Bataan to Quezon City, which is around 80 kilometers, using a truck should take approximately 2.5 hours. This estimate assumes paved roads for most of the journey. However, you may experience delays due to heavy traffic as you approach Quezon City, especially during the morning and afternoon rush hours. Consider allocating additional time for potential delays."
User Prompt: "I need help calculating the transport cost of delivering 2 tons of tomatoes from Cavite to Manila. What are the key factors I should consider in my cost calculation?"

Model Response: "When calculating transport costs for delivering tomatoes from Cavite to Manila (approximately 40 kilometers), the key factors to consider are fuel costs, mode of transport, and road conditions. A truck or van would be suitable for this transport, with an estimated fuel cost of around ₱59 per liter. For 2 tons of tomatoes, the transport cost would depend on the fuel consumption of the vehicle, which on average is about 8-10 kilometers per liter for a truck. Don’t forget to factor in any loading and unloading time, as well as possible traffic conditions on the way to Manila."

"""

In [ ]:
struct = [{'role' : 'system', 'content' : system_prompt}]

In [ ]:
openai.api_key = ""

In [ ]:
dataframed = pd.read_csv('https://raw.githubusercontent.com/erwinjamescodes/Day-4-AI-First-Dataset/refs/heads/main/farm-to-market.csv')

In [ ]:
dataframed.head()

,farmer_id,farmer_name,farmer_age,farmer_gender,farm_location,farm_size_hectares,primary_crop,annual_yield_tons,experience_years,transport_id,...,selling_price_per_kg,profit_margin,payment_method,payment_terms_days,weather_conditions,season,fuel_cost_per_liter,transport_frequency_per_week,storage_facility,delivery_window_hours
0,F001,Juan Dela Cruz,45,Male,"Barangay San Isidro, Tarlac",2.5,Rice,5.0,20,T001,...,20,0.25,Cash,0,Sunny,Harvest,58.0,2,Yes,08:00-17:00
1,F002,Maria Santos,52,Female,"Barangay San Juan, Nueva Ecija",3.0,Corn,6.0,25,T002,...,18,0.30,Bank Transfer,3,Rainy,Harvest,60.0,1,No,09:00-18:00
2,F003,Pedro Flores,38,Male,"Barangay Santo Niño, Laguna",1.8,Tomatoes,4.2,12,T003,...,22,0.28,Cash,0,Cloudy,Harvest,55.0,3,Yes,07:00-16:00
3,F004,Ana Reyes,47,Female,"Barangay San Roque, Batangas",3.5,Bananas,10.0,22,T004,...,15,0.35,Cash,0,Sunny,Harvest,NaN,2,No,06:00-12:00
4,F005,Carlos Tan,55,Male,"Barangay Mabini, Quezon",2.1,Coconuts,8.0,30,T005,...,25,0.22,Bank Transfer,2,Sunny,Harvest,58.0,1,Yes,10:00-18:00


In [ ]:
dataframed['combined'] = dataframed.apply(lambda row : " ".join(row.values.astype(str)), axis = 1)

In [ ]:
documents = dataframed['combined'].tolist()

In [ ]:
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in documents]

In [ ]:
embedding_dim = len(embeddings[0])

In [ ]:
embeddings_np = np.array(embeddings).astype('float32')

In [ ]:
index = faiss.IndexFlatL2(embedding_dim)

In [ ]:
index.add(embeddings_np)

In [ ]:
user_message = "Can you estimate the time it will take for a truck to transport 3 tons of papayas from Bataan to Quezon City, taking into account the usual road conditions?"

In [ ]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')
_, indices = index.search(query_embedding_np, 5)
retrieved_docs = [documents[i] for i in indices[0]]
context = ' '.join(retrieved_docs)
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [ ]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [ ]:
print(response)

To estimate the time it will take for a truck to transport 3 tons of papayas from Bataan to Quezon City, we first need to consider the distance and the typical road conditions.

The distance from Bataan to Quezon City is approximately 80 kilometers. Given that the roads are mostly paved, we can estimate the travel time based on average speeds for trucks on such roads.

1. **Average Speed**: For a truck on paved roads, an average speed of around 40-50 km/h is reasonable, but since you may encounter traffic as you approach Quezon City, we will use a more conservative estimate of 40 km/h.

2. **Travel Time Calculation**: 
   - Distance: 80 kilometers
   - Speed: 40 km/h

   \[
   \text{Travel Time} = \frac{\text{Distance}}{\text{Speed}} = \frac{80 \text{ km}}{40 \text{ km/h}} = 2 \text{ hours}
   \]

3. **Traffic Considerations**: Given that you will be entering Quezon City, it is prudent to add extra time for potential traffic congestion. During peak hours, this could add approximately 3